In [85]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re

pd.set_option('display.max_columns', None)

Load, clean, and write back to disk.

In [86]:
# Read files from directory.
files = [x for x in os.listdir('data') if '_data_only' in x]

# Sort by earlier year first.
files = list(sorted(files, key=lambda x: int(x[:4])))

print('File names:', files)

# Load into memory.
dfs = [
    pd.read_csv('data/' + file_name, delimiter=',', quotechar='"', encoding='ISO-8859-1')
    for file_name in files
]

# Clean up column names for consistency. 
for i, dframe in enumerate(dfs):
    dfs[i].columns = [
        re.sub(r"_+", "_", x \
            .strip() \
            .lower() \
            .replace(' ', '_') \
            .replace('\n', '_') \
            .replace('ï»¿agency', 'agency')
        )
        for x in dframe.columns
    ]

# Check for columns that are not shared between dataframes.
cols = [set(dframe.columns) for dframe in dfs]

cols_not_shared = []
common_columns_across_dfs = set.intersection(*cols)
for i, col_set in enumerate(cols):
    other_cols = set.union(*(cols[: i] + cols[i + 1 :])) 
    unique_to_this_df = col_set - other_cols  
    missing_from_this_df = other_cols - col_set  
    cols_not_shared.extend(unique_to_this_df)

for i, df in enumerate(dfs):
    for col in cols_not_shared:
        if col in df.columns:
            print(f'Dropping {col} because col is not shared.')
            dfs[i] = df.drop(col, axis=1)

# Add year to each dataframe.
for i, year in enumerate(range(2018, 2023)):
    dfs[i]['year'] = year

# Combine into one dataframe.
df = pd.DataFrame()
for dframe in dfs:
    df = pd.concat([df, dframe], axis=0)

len_df = len(df)
df = df[df['any_data_questionable?'] != 'Yes']
print(f'Dropped {len_df - len(df)} rows because data was questionable.')

# Drop cols with many NaN values.
MIN_NANS_PERCENT = .15  # Each year is roughly 20% of the total rows
for col in df.columns:
    if df[col].isna().sum() / len(df) > MIN_NANS_PERCENT:
        print(f'Dropping {col} because % NaN is: {df[col].isna().sum() / len(df)}')
        df = df.drop(col, axis=1)

# Remove extra characters from numeric columns and cast to float.
numeric_cols = [
    'cost_per_hour',
    'primary_uza_population',
    'vehicle_revenue_miles',
    'passenger_miles',
    'vehicle_revenue_hours',
    'unlinked_passenger_trips',
    'total_operating_expenses',
    'fare_revenues_earned',
    'cost_per_passenger_mile',
    'cost_per_passenger',
    'cost_per_hour',
    'fare_revenues_per_total_operating_expense_(recovery_ratio)',
    'primary_uza_population',
    'agency_voms',
    'mode_voms',
    'fare_revenues_per_unlinked_passenger_trip'
]

for col in numeric_cols:
    df[col] = df[col].apply(lambda x: (
        str(x) \
            .replace('$', '') \
            .replace('(', '') \
            .replace(')', '') \
            .replace(',', '')
            .strip()
    ))
    df[col] = df[col].astype(float)

df.reset_index(drop=True, inplace=True)

display(df.sample(1, random_state=538))

df.to_csv('data/clean_metrics_18_22.csv', index=False)

File names: ['2018_Metrics_data_only.csv', '2019_Metrics_data_only.csv', '2020-Metrics_data_only.csv', '2021 Metrics_static_data_only.csv', '2022_NTD_Annual_Data_-_Metrics_20231102_data_only.csv']
Dropping 0 because col is not shared.
Dropping uace_code because col is not shared.
Dropping passengers_per_vehicle_revenue_hour because col is not shared.
Dropping uza_name because col is not shared.
Dropped 443 rows because data was questionable.
Dropping legacy_ntd_id because % NaN is: 0.2714436015283238
Dropping ratios: because % NaN is: 1.0
Dropping fare_revenues_per_unlinked_passenger_trip_questionable because % NaN is: 0.9957361980176089
Dropping fare_revenues_per_total_operating_expense_(recovery_ratio)_questionable because % NaN is: 1.0
Dropping cost_per_hour_questionable because % NaN is: 0.9985602746552965
Dropping passengers_per_hour because % NaN is: 0.20438562489617365
Dropping passengers_per_hour_questionable because % NaN is: 0.9951270834486959
Dropping cost_per_passenger_ques

,agency,city,state,ntd_id,organization_type,reporter_type,primary_uza_population,agency_voms,mode,tos,mode_voms,fare_revenues_per_unlinked_passenger_trip,fare_revenues_per_total_operating_expense_(recovery_ratio),cost_per_hour,cost_per_passenger,cost_per_passenger_mile,fare_revenues_earned,total_operating_expenses,unlinked_passenger_trips,vehicle_revenue_hours,passenger_miles,vehicle_revenue_miles,year
8777,Fairmont Marion County Transit Authority,Fairmont,WV,3R05-30149,Independent Public Agency or Authority of Tran...,Rural Reporter,0.0,23.0,MB,DO,12.0,0.66,0.07,107.31,10.11,0.0,98648.0,1508329.0,149140.0,14056.0,0.0,282978.0,2020


In [87]:
for year in df.year.unique():
    for col in df.columns:
        is_na = df[df.year == year][col].isna().sum()
        is_na_percent = round(is_na / len(df[df.year == year]) * 100, 2)
        if is_na_percent > 1:
            print(f"{year} NaNs: {is_na_percent}% - {col}")

2022 NaNs: 38.74% - primary_uza_population
